# Spam detection

Here I wil try several aproaches for fast and accurate with high f1-score spam detection

In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import pandas as pd

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.nn import functional as F

In [2]:
from datasets import load_dataset

# ru_spam_dataset_v6 = load_dataset("RUSpam/spam_dataset_v6")

# en_telegram_spam_dataset = load_dataset("thehamkercat/telegram-spam-ham")

# en_sms_spam_dataset = load_dataset("ucirvine/sms_spam")

# en_spam_dataset = load_dataset("Deysi/spam-detection-dataset")


# Preprocessing

Let's merge datasets, then filter for duplicates, get statistical analysis

## Merging and filtering

In [3]:
# ru_spam_dataset_v6

In [4]:
# ru_spam_dataset_v6 = ru_spam_dataset_v6['train'].to_pandas()

In [5]:
# ru_spam_dataset_v6.head()

In [6]:
# ru_spam_dataset_v6['label'].unique()

In [7]:
# en_telegram_spam_dataset

In [8]:
# en_telegram_spam_dataset = en_telegram_spam_dataset['train'].to_pandas()

In [9]:
# en_telegram_spam_dataset.head()

In [10]:
# en_telegram_spam_dataset['text_type'].unique()

In [11]:
# en_sms_spam_dataset

In [12]:
# en_sms_spam_dataset = en_sms_spam_dataset['train'].to_pandas()

In [13]:
# en_sms_spam_dataset.head()

In [14]:
# en_sms_spam_dataset['label'].unique()

In [15]:
# en_spam_dataset

In [16]:
# en_spam_dataset = pd.concat([en_spam_dataset['train'].to_pandas(), en_spam_dataset['test'].to_pandas()], ignore_index=True)

In [17]:
# en_spam_dataset.head()

In [18]:
# en_spam_dataset['label'].unique()

Now let's rename columns of all datasets to one variant and merge

In [19]:
# class SPAM_DATASET_INFO:
#     def __init__(self):
#         self.columns = {'message_column':'message', 'label_column': 'is_spam'}

Also let's encode label: 1 - spam, 0 - not

In [20]:
# SPAM_DATASET_INFO_ = SPAM_DATASET_INFO()

In [21]:
# ru_spam_dataset_v6.rename(columns={'message': SPAM_DATASET_INFO_.columns['message_column'], 'label': SPAM_DATASET_INFO_.columns['label_column']}, inplace=True)

In [22]:
# ru_spam_dataset_v6['is_spam'].unique()

In [23]:
# en_telegram_spam_dataset.rename(columns={'text': SPAM_DATASET_INFO_.columns['message_column'], 'text_type': SPAM_DATASET_INFO_.columns['label_column']}, inplace=True)

In [24]:
# en_telegram_spam_dataset.head()

In [25]:
# en_telegram_spam_dataset['is_spam'].unique()

In [26]:
# en_telegram_spam_dataset['is_spam'] = en_telegram_spam_dataset[SPAM_DATASET_INFO_.columns['label_column']].apply(lambda x: 0 if x=='spam' else 1)

In [27]:
# en_telegram_spam_dataset['is_spam'].unique()

In [28]:
# en_sms_spam_dataset.head()

In [29]:
# en_sms_spam_dataset.rename(columns={'sms': SPAM_DATASET_INFO_.columns['message_column'], 'label': SPAM_DATASET_INFO_.columns['label_column']}, inplace=True)

In [30]:
# en_sms_spam_dataset['is_spam'].unique()

In [31]:
# en_sms_spam_dataset.head()

In [32]:
# en_spam_dataset.rename(columns={'text': SPAM_DATASET_INFO_.columns['message_column'], 'label': SPAM_DATASET_INFO_.columns['label_column']}, inplace=True)

In [33]:
# en_spam_dataset.head()

In [34]:
# en_spam_dataset['is_spam'].unique()

In [35]:
# en_spam_dataset['is_spam'] = en_spam_dataset['is_spam'].apply(lambda x: 1 if x=='spam' else 0)

In [36]:
# en_spam_dataset['is_spam'].unique()

In [37]:
# spam_dataset_df = pd.concat([ru_spam_dataset_v6, en_telegram_spam_dataset, en_sms_spam_dataset, en_spam_dataset], ignore_index=True)

In [38]:
# spam_dataset_df

Let's count rows

In [39]:
# spam_dataset_df.info()

In [40]:
# Drop duplicates based on column 'A'
# spam_dataset_df.drop_duplicates(subset=['message']).shape

In [41]:
# spam_dataset_df_filtered = spam_dataset_df.drop_duplicates(subset=['message'])

In [42]:
# plt.hist(spam_dataset_df['message'].apply(lambda x: len(x)), bins=30)
# plt.title('not tokenized messages length distribution')
# plt.yscale('log');

In [43]:
# TELEGRAM_MAX_MESSAGE_LEN = 4096

The target - train efficient spam detector on for telegram, that's why need to cut dataset by messages length

In [44]:
# spam_dataset_df[spam_dataset_df['message'].apply(lambda x: len(x)>TELEGRAM_MAX_MESSAGE_LEN)]

In [45]:
# spam_dataset_df.drop(spam_dataset_df[spam_dataset_df['message'].apply(lambda x: len(x)>TELEGRAM_MAX_MESSAGE_LEN)].index, inplace=True)

Split data on train, test, val

In [46]:
# spam_dataset_df[spam_dataset_df['message'].apply(lambda x: len(x)>TELEGRAM_MAX_MESSAGE_LEN)]

# Let's finetune some models

In [47]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

2024-08-18 18:53:22.516117: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 18:53:22.516214: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 18:53:22.684094: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Make datasets

In [48]:
from datasets import Dataset

In [49]:
# spam_dataset_df.rename(columns={'message':'text', 'is_spam': 'labels'}, inplace=True)

In [50]:
# spam_dataset_df.head()

In [51]:
# train_df, temp_df = train_test_split(spam_dataset_df, test_size=0.3, stratify=spam_dataset_df['labels'])
# eval_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['labels'])

# # Преобразование pandas DataFrame в Hugging Face Dataset
# train_dataset = Dataset.from_pandas(train_df)
# eval_dataset = Dataset.from_pandas(eval_df)
# test_dataset = Dataset.from_pandas(test_df)

In [52]:
# # Токенизация данных
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

Let's add more dataset specific tokens to tokenizer

In [53]:
# new_tokens = set(' '.join(spam_dataset_df['text']))

In [54]:
# num_added_tokens = tokenizer.add_tokens(list(new_tokens))

In [55]:
# num_added_tokens

In [56]:
# def tokenize_function(examples):
#     # Токенизация входных данных
#     return tokenizer(examples['text'], padding='max_length', truncation=True)

# # Применение токенизации к каждому датасету
# train_dataset = train_dataset.map(tokenize_function, batched=True)
# eval_dataset = eval_dataset.map(tokenize_function, batched=True)
# test_dataset = test_dataset.map(tokenize_function, batched=True)

In [57]:
# train_dataset

In [58]:
# train_dataset.save_to_disk("train_dataset.hf")
# eval_dataset.save_to_disk("eval_dataset.hf")
# test_dataset.save_to_disk("test_dataset.hf")

In [59]:
# !zip -r train_dataset.zip train_dataset.hf
# !zip -r eval_dataset.zip eval_dataset.hf
# !zip -r test_dataset.zip test_dataset.hf

In [60]:
# tokenizer.save_pretrained('pretrained_tokenizer')

In [61]:
# !zip -r pretrained_tokenizer.zip pretrained_tokenizer

In [62]:
# from numba import cuda
# cuda.select_device(0)
# cuda.close()
# cuda.select_device(0)

In [63]:
from datasets import load_from_disk

In [64]:
train_dataset = load_from_disk('/kaggle/input/spam-detector-distilbert/train_dataset/train_dataset.hf')
eval_dataset = load_from_disk('/kaggle/input/spam-detector-distilbert/eval_dataset/eval_dataset.hf')
test_dataset = load_from_disk('/kaggle/input/spam-detector-distilbert/test_dataset/test_dataset.hf')

In [65]:
train_dataset

Dataset({
    features: ['text', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 495223
})

In [66]:
tokenizer = DistilBertTokenizer.from_pretrained('/kaggle/input/spam-detector-distilbert/pretrained_tokenizer/pretrained_tokenizer')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [67]:
# torch.cuda.empty_cache()

In [68]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [69]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=2)

# Resize the model’s token embeddings to accommodate new tokens
model.resize_token_embeddings(len(tokenizer))

# Define training arguments with W&B disabled
training_args = TrainingArguments(
    output_dir='./results',
    save_total_limit=3,
    save_steps=1000,
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2.2,
    weight_decay=0.01,
    report_to=None,  # Disable W&B logging
    no_cuda=False  # Ensures GPU is used if available
    
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train model
trainer.train()

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.049500,0.046352
2,0.027700,0.038779


TrainOutput(global_step=17024, training_loss=0.0507376146768885, metrics={'train_runtime': 31798.2087, 'train_samples_per_second': 34.263, 'train_steps_per_second': 0.535, 'total_flos': 1.443256152488878e+17, 'train_loss': 0.0507376146768885, 'epoch': 2.200051692943913})

In [70]:
# Оценка модели
# eval_results = trainer.evaluate()

# print("Evaluation results:", eval_results)

# Сохранение модели
model.save_pretrained('./saved_model')

In [71]:
!zip saved_model.zip -r saved_model

  adding: saved_model/ (stored 0%)
  adding: saved_model/config.json (deflated 46%)
  adding: saved_model/model.safetensors (deflated 7%)
